In [1]:
# %% [markdown]
# # Script gộp danh sách các file JS
# Script này sẽ đọc các file chứa array object `{ text, link }`,
# loại bỏ đuôi `.md` ở link, và gộp chung lại thành một file `searchItems.js`.

# %%
import os
import re
import glob

def merge_js_files(input_files, output_file):
    """
    Hàm đọc và gộp danh sách các file .js

    Args:
        input_files (list): Danh sách đường dẫn tới các file .js đầu vào
        output_file (str): Đường dẫn file .js đầu ra
    """
    all_items = []

    # Biểu thức Regex để bắt các object có định dạng: { text: "...", link: "..." }
    # Hỗ trợ cả dấu nháy đơn (') và nháy kép (")
    pattern = re.compile(r'\{\s*text\s*:\s*(["\'])(.*?)\1\s*,\s*link\s*:\s*(["\'])(.*?)\3\s*\}')

    for file_path in input_files:
        if not os.path.exists(file_path):
            print(f"⚠️ Cảnh báo: Không tìm thấy file {file_path}")
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

            # Tìm tất cả các khớp (matches) trong file
            matches = pattern.findall(content)

            for match in matches:
                # match[1] là giá trị của text, match[3] là giá trị của link
                text_value = match[1]
                link_value = match[3]

                # Xóa đuôi .md nếu có
                if link_value.endswith('.md'):
                    link_value = link_value[:-3]

                all_items.append({
                    "text": text_value,
                    "link": link_value
                })

        print(f"✅ Đã đọc file: {file_path} (Tìm thấy {len(matches)} items)")

    # Tiến hành ghi ra file gộp (output)
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('export const searchItems = [\n')

        for i, item in enumerate(all_items):
            # Escape các ký tự nháy kép (") trong text để tránh gây lỗi cú pháp JS
            safe_text = item["text"].replace('"', '\\"')
            safe_link = item["link"]

            # Tạo chuỗi format cho từng dòng
            line = f'    {{ text: "{safe_text}", link: "{safe_link}" }}'

            # Thêm dấu phẩy ở cuối nếu chưa phải item cuối cùng
            if i < len(all_items) - 1:
                line += ','

            f.write(line + '\n')

        f.write('];\n')

    print(f"\n🎉 HOÀN TẤT! Đã gộp thành công {len(all_items)} items vào file: '{output_file}'")

# %%
# ==========================================
# CẤU HÌNH VÀ CHẠY THỬ NGHIỆM
# ==========================================

# CÁCH 1: Liệt kê thủ công từng file
input_files = [
    '../docs/kinhtrungbo/nanamoli-bodhi-vi/meta/filelist.js',
    '../docs/kinhtrungbo/thichminhchau/meta/filelist.js',
    # Thêm đường dẫn các file của bạn vào đây...
]

# CÁCH 2: Quét toàn bộ file .js trong một thư mục (Bỏ comment dòng dưới để dùng)
# input_files = glob.glob('./thu_muc_chua_js/*.js')

# Tên file xuất ra
output_file = '../docs/search-items.js'

# Gọi hàm
merge_js_files(input_files, output_file)

✅ Đã đọc file: ../docs/kinhtrungbo/nanamoli-bodhi-vi/meta/filelist.js (Tìm thấy 175 items)
✅ Đã đọc file: ../docs/kinhtrungbo/thichminhchau/meta/filelist.js (Tìm thấy 155 items)

🎉 HOÀN TẤT! Đã gộp thành công 330 items vào file: '../docs/search-items.js'
